In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

In [2]:
DATABASE_TYPE="postgresql"
USERNAME="adminff"
PASSWORD="ffmodeling"
HOST="localhost"
PORT=5433
DATABASE_NAME="ff"
FIRE_SUBDIVISION_TABLE = "fire_division_dataset"

In [3]:
engine = create_engine(f"{DATABASE_TYPE}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}")

In [4]:
query_fire = """select f."YEAR", f."MONTH", f."DAY", f."REP_DATE", f."CALC_HA", f."CAUSE", f."geometry" from fire f"""
query_subdivision = """select * from subdivision_dataset sd"""

In [5]:
fire_gdf = gpd.read_postgis(
    sql=query_fire, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326",)  

In [6]:
fire_gdf

,YEAR,MONTH,DAY,REP_DATE,CALC_HA,CAUSE,geometry
0,2004,6,23,2004-06-23,1370.507344,L,"POLYGON Z ((-124.61060 53.08084 0.00000, -124...."
1,2004,6,23,2004-06-23,520.796287,L,"POLYGON Z ((-124.48262 53.06162 0.00000, -124...."
2,2004,6,20,2004-06-20,268.290572,L,"POLYGON Z ((-125.12760 52.13023 0.00000, -125...."
3,2004,6,21,2004-06-21,20506.415129,L,"POLYGON Z ((-125.75419 52.27968 0.00000, -125...."
4,2004,6,22,2004-06-22,2408.587142,L,"POLYGON Z ((-126.26477 52.99880 0.00000, -126...."
...,...,...,...,...,...,...,...
59533,1994,0,0,NaT,12377.902873,U,"MULTIPOLYGON Z (((-129.11700 62.68432 0.00000,..."
59534,2004,0,0,NaT,200.441770,U,"POLYGON Z ((-128.90697 62.59442 0.00000, -128...."
59535,2005,0,0,NaT,6.040325,U,"POLYGON Z ((-129.63559 62.91861 0.00000, -129...."
59536,2009,0,0,NaT,73.532206,U,"POLYGON Z ((-128.95992 62.32435 0.00000, -128...."


In [8]:
division_gdf = gpd.read_postgis(
    sql=query_subdivision, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326",)  

In [9]:
division_gdf

,cid,geometry
0,0,"MULTIPOLYGON (((-139.53267 69.53197, -139.5336..."
1,1,"MULTIPOLYGON (((-59.17563 47.55863, -59.17579 ..."
2,2,"MULTIPOLYGON (((-112.70518 67.67319, -112.7050..."
3,3,"MULTIPOLYGON (((-82.52584 42.47577, -82.52580 ..."
4,4,"POLYGON ((-112.31319 55.39631, -112.31267 55.3..."
...,...,...
62,62,"POLYGON ((-121.20810 50.56893, -121.20824 50.5..."
63,63,"POLYGON ((-118.79457 54.58827, -118.74824 54.5..."
64,64,"MULTIPOLYGON (((-121.00702 49.65631, -121.0066..."
65,65,"MULTIPOLYGON (((-80.76446 45.82386, -80.76447 ..."


In [10]:
fire_in_subdivision_gdf = gpd.sjoin(
    fire_gdf,
    division_gdf,
    how="inner",
    predicate="within").drop("index_right", axis=1)

In [11]:
fire_in_subdivision_gdf = fire_in_subdivision_gdf.rename(
    {
        "YEAR": "year",
        "MONTH": "month",
        "DAY": "day",
        "REP_DATE": "start_date",
        "CALC_HA": "area_burnt_ha",
        "CAUSE": "cause",
        "cid": "division_id"
    },
    axis = 1
)

In [12]:
# group to indexes
fire_in_subdivision_gdf = fire_in_subdivision_gdf.groupby(["division_id","start_date","area_burnt_ha","cause",]).first().reset_index()

In [13]:
# drop duplicates
fire_in_subdivision_gdf.drop_duplicates(inplace=True)

In [14]:
fire_in_subdivision_gdf.to_postgis(name=FIRE_SUBDIVISION_TABLE, con=engine, if_exists='replace', index=False)

/home/mutakabbir/Projects/Lightning/.venv/lib/python3.10/site-packages/geopandas/io/sql.py:415: UserWarning: Could not parse CRS from the GeoDataFrame. Inserting data without defined CRS.
  srid = _get_srid_from_crs(gdf)


In [18]:
with engine.connect() as con:
    con.execute(f'ALTER TABLE {FIRE_SUBDIVISION_TABLE} ADD PRIMARY KEY ("division_id","start_date","area_burnt_ha","cause");')

In [ ]:
# # Operation too long
# fire_overlaps_subdivision_gdf = gpd.sjoin(
#     fire_not_in_sub_division,
#     division_gdf,
#     how="inner",
#     predicate="overlaps").drop("index_right", axis=1)

In [19]:
fire_in_subdivision_gdf

,division_id,start_date,area_burnt_ha,cause,year,month,day,geometry
0,0,1960-05-10,6422.863612,L,1960,5,10,"POLYGON Z ((-136.74856 66.77644 0.00000, -136...."
1,0,1960-06-12,679.814663,L,1960,6,12,"POLYGON Z ((-136.63157 65.80181 0.00000, -136...."
2,0,1960-07-15,329.731143,H,1960,7,15,"POLYGON Z ((-136.87935 67.32493 0.00000, -136...."
3,0,1960-07-20,559.296563,H,1960,7,20,"POLYGON Z ((-124.77179 60.06496 0.00000, -124...."
4,0,1960-08-03,724.944073,H,1960,8,3,"POLYGON Z ((-133.87588 60.12462 0.00000, -133...."
...,...,...,...,...,...,...,...,...
46478,66,2019-06-02,1.046225,H,2019,6,2,"POLYGON Z ((-114.95344 51.21220 0.00000, -114...."
46479,66,2020-09-04,2417.256996,H,2020,9,4,"MULTIPOLYGON Z (((-115.26752 51.29490 0.00000,..."
46480,66,2020-10-05,2.773375,H,2020,10,5,"POLYGON Z ((-114.90744 51.12062 0.00000, -114...."
46481,66,2020-10-11,18.277458,H,2020,10,11,"POLYGON Z ((-114.74050 51.20659 0.00000, -114...."
